## 1. Libraries and settings

In [ ]:
import numpy as np
import random
import pandas as pd 
from pylab import mpl, plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline
import torch
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import vectorbtpro as vbt

from datetime import date

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter("ignore", UserWarning)

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

# #hparams
timestep = 80
# # Update these dimensions based on your dataset

# hidden_dim = 32
# num_layers = 2

# num_epochs = 200
# learning_rate=0.01
# step_size=30
# gamma=0.9

# dropout_rate=0.2
# print_epochs = 2

In [ ]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

df['signal'] = df['signal'].replace({'SignalNone': 1, 'SignalLong': 2, 'SignalShort': 0})
df.fillna(method='pad');


In [ ]:
data = vbt.Data.from_data(df)
features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))
data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')

open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = data.data['symbol'].drop('signal', axis=1).columns.tolist()


X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

X.columns = X.columns.astype(str)


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# First, split your data into a training+validation set and a separate test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3)

# Then, split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)  # 0.2 here means 20% of the original data, or 25% of the training+validation set



# Create a scaler instance
scaler = StandardScaler()

# Fit the scaler to your data and transform
X_train_scaled = scaler.fit_transform(X_train)

# Create a DataFrame from the scaled data with the same index and columns
X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

# Fit the scaler to your data and transform
X_test_scaled = scaler.fit_transform(X_test)

# Create a DataFrame from the scaled data with the same index and columns
X_test_scaled_df = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

# Fit the scaler to your data and transform
X_val_scaled = scaler.fit_transform(X_val)

# Create a DataFrame from the scaled data with the same index and columns
X_val_scaled_df = pd.DataFrame(X_val_scaled, index=X_val.index, columns=X_val.columns)



In [ ]:
if np.array_equal(X_train_scaled_df.values, X_train_scaled):
    print('Success')

In [ ]:


def create_sequences(input_data, sequence_length):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - sequence_length):
        seq = input_data[i:(i + sequence_length)]
        sequences.append(seq)
    return np.array(sequences)

# Assuming X_train_scaled_df and X_test_scaled_df are already scaled and are DataFrames
X_train_list = create_sequences(X_train_scaled_df.values, timestep)
X_test_list = create_sequences(X_test_scaled_df.values, timestep)
X_val_list = create_sequences(X_val_scaled_df.values, timestep)

y_train_seq_ar = y_train[timestep:]
y_test_seq_ar = y_test[timestep:]
y_val_seq_ar = y_val[timestep:]


x_train = np.array(X_train_list)
x_test = np.array(X_test_list)  
x_val = np.array(X_val_list)  

y_train_seq = np.array(y_train_seq_ar)
y_test_seq = np.array(y_test_seq_ar)
y_val_seq = np.array(y_val_seq_ar)



# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Move tensors to the MPS device
x_train_tensor = torch.from_numpy(x_train).float().to(device)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_train_tensor = torch.from_numpy(y_train_seq).long().to(device)
y_test_tensor = torch.from_numpy(y_test_seq).long().to(device)



In [ ]:

from sklearn.utils.class_weight import compute_class_weight



# Convert y_train to a numpy array if it's a tensor

if isinstance(y_train_seq, torch.Tensor):
    y_train_seq_np = y_train_seq.cpu().numpy()
else:
    y_train_seq_np = y_train_seq  # Assuming y_train_seq is already a numpy array or similar

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_seq_np), y=y_train_seq_np)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)


# Move class weights to the same device as your model and data
class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'


In [ ]:

import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

num_epochs = 150
input_dim = 175  # Number of features
output_dim = 3  # Number of classes

class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layer
        # The input dimension is twice the hidden_dim because it's bidirectional
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # times 2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # times 2 for bidirectional
        
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Apply dropout to the output of the LSTM
        out = self.dropout(out)
        
        # Concatenate the hidden states from both directions
        out = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
        
        # Pass the concatenated hidden states to the fully connected layer
        out = self.fc(out)
        
        return out




# Define the objective function that Optuna will optimize
def objective(trial):
    # Hyperparameters to be optimized by Optuna
    hidden_dim = trial.suggest_categorical('hidden_dim', [16, 32, 64, 128])
    num_layers = trial.suggest_int('num_layers', 1, 3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    step_size = trial.suggest_int('step_size', 10, 100)
    gamma = trial.suggest_uniform('gamma', 0.85, 0.99)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    
    # Update the model with new hyperparameters
    model = BiLSTMClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim, dropout_rate=dropout_rate)
    model.to(device)
    
    # Convert y_train_seq and y_val_seq for the current trial
    y_train_tensor = torch.from_numpy(y_train_seq).long().to(device)  # Convert y_train_seq
    y_val_tensor = torch.from_numpy(y_val_seq).long().to(device)  # Convert y_val_seq
    x_val_tensor = torch.from_numpy(x_val).float().to(device)  # Convert y_val_seq

    # Loss function and optimizers
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        # Validation step
        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                val_output = model(x_val_tensor)  # You need to create a validation set 'x_val'
                val_loss = criterion(val_output, y_val_tensor)  # You need to create a validation set 'y_val'
            model.train()
    
    # Objective value to minimize (could be the validation loss)
    return val_loss.item()

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Specify the number of trials

# Get the best hyperparameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

# You can now retrain your model with the best parameters or analyze the results


In [ ]:
# # Adjust the figure size
# plt.figure(figsize=(6, 3))

# # Plot the training loss
# plt.plot(hist, label="Training loss")
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
with torch.no_grad():
    y_test_pred = model(x_test)
    # Convert logits to probabilities
    probabilities = torch.softmax(y_test_pred, dim=1)
    # Get the predicted class labels
    _, predicted_labels = torch.max(probabilities, 1)
    # Move the tensor to CPU and then convert to numpy
    predicted_labels_numpy = predicted_labels.cpu().numpy()
    print(len(predicted_labels_numpy))


In [ ]:
df_split = data.data['symbol'][-len(predicted_labels_numpy):].copy()
df_split.loc[:, "signal"] = predicted_labels_numpy
signal = df_split['signal']
entries = signal == 2
exits = signal == 0

In [ ]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [ ]:
pf.plot(settings=dict(bm_returns=False)).show()

In [ ]:
pf.stats()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Convert tensors to numpy arrays for use with Scikit-Learn
true_labels = y_test.cpu().numpy()
pred_labels = predicted_labels.cpu().numpy()

precision = precision_score(true_labels, pred_labels, average='macro')  # 'macro' for unweighted mean
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')
conf_matrix = confusion_matrix(true_labels, pred_labels)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print('Confusion Matrix:\n', conf_matrix)